Data Cleaner
==============

**Author:** *Nicolas Haase*

## Data Import

In [1]:
import pandas as pd

# Set view options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
# Load flight data
df = pd.read_csv('/Users/nico/Developments/strategic-flight-prediction/data/flight_data.csv')

/var/folders/k_/7tyv0nps0tn9zv3mhhdms5y80000gn/T/ipykernel_40381/1314337724.py:2: DtypeWarning: Columns (48,69,76,77,84,85,92) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/nico/Developments/strategic-flight-prediction/data/flight_data.csv')


In [3]:
# Load airport coordinates data
airport_coordinates = pd.read_csv('../data/airport_coordinates.csv')

In [4]:
# Load weather data
weather = pd.read_csv('../data/weather/atl_weather_hourly_averages.csv')
monthly_weather = pd.read_csv('../data/weather/atl_weather_monthly_averages.csv')

## Data Information

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13453022 entries, 0 to 13453021
Columns: 110 entries, Year to Unnamed: 109
dtypes: float64(69), int64(20), object(21)
memory usage: 11.0+ GB


In [6]:
df.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,Dest,DestCityName,DestState,DestStateFips,DestStateName,DestWac,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,Cancelled,CancellationCode,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,FirstDepTime,TotalAddGTime,LongestAddGTime,DivAirportLandings,DivReachedDest,DivActualElapsedTime,DivArrDelay,DivDistance,Div1Airport,Div1AirportID,Div1AirportSeqID,Div1WheelsOn,Div1TotalGTime,Div1LongestGTime,Div1WheelsOff,Div1TailNum,Div2Airport,Div2AirportID,Div2AirportSeqID,Div2WheelsOn,Div2TotalGTime,Div2LongestGTime,Div2WheelsOff,Div2TailNum,Div3Airport,Div3AirportID,Div3AirportSeqID,Div3WheelsOn,Div3TotalGTime,Div3LongestGTime,Div3WheelsOff,Div3TailNum,Div4Airport,Div4AirportID,Div4AirportSeqID,Div4WheelsOn,Div4TotalGTime,Div4LongestGTime,Div4WheelsOff,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Unnamed: 109
0,2021,3,8,5,4,2021-08-05,OH,20397,OH,N525AE,5574,10599,1059904,30599,BHM,"Birmingham, AL",AL,1,Alabama,51,11057,1105703,31057,CLT,"Charlotte, NC",NC,37,North Carolina,36,1914,1906.0,-8.0,0.0,0.0,-1.0,1900-1959,17.0,1923.0,2128.0,9.0,2151,2137.0,-14.0,0.0,0.0,-1.0,2100-2159,0.0,NaN,0.0,97.0,91.0,65.0,1.0,350.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021,3,8,6,5,2021-08-06,OH,20397,OH,N708PS,5574,10599,1059904,30599,BHM,"Birmingham, AL",AL,1,Alabama,51,11057,1105703,31057,CLT,"Charlotte, NC",NC,37,North Carolina,36,1914,1948.0,34.0,34.0,1.0,2.0,1900-1959,14.0,2002.0,2159.0,23.0,2151,2222.0,31.0,31.0,1.0,2.0,2100-2159,0.0,NaN,0.0,97.0,94.0,57.0,1.0,350.0,2,0.0,0.0,0.0,0.0,31.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021,3,8,8,7,2021-08-08,OH,20397,OH,N712PS,5574,10599,1059904,30599,BHM,"Birmingham, AL",AL,1,Alabama,51,11057,1105703,31057,CLT,"Charlotte, NC",NC,37,North Carolina,36,1914,1913.0,-1.0,0.0,0.0,-1.0,1900-1959,11.0,1924.0,2119.0,29.0,2151,2148.0,-3.0,0.0,0.0,-1.0,2100-2159,0.0,NaN,0.0,97.0,95.0,55.0,1.0,350.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021,3,8,9,1,2021-08-09,OH,20397,OH,N503AE,5574,10599,1059904,30599,BHM,"Birmingham, AL",AL,1,Alabama,51,11057,1105703,31057,CLT,"Charlotte, NC",NC,37,North Carolina,36,1914,1906.0,-8.0,0.0,0.0,-1.0,1900-1959,11.0,1917.0,2116.0,15.0,2151,2131.0,-20.0,0.0,0.0,-2.0,2100-2159,0.0,NaN,0.0,97.0,85.0,59.0,1.0,350.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021,3,8,10,2,2021-08-10,OH,20397,OH,N513AE,5574,10599,1059904,30599,BHM,"Birmingham, AL",AL,1,Alabama,51,11057,1105703,31057,CLT,"Charlotte, NC",NC,37,North Carolina,36,1914,2008.0,54.0,54.0,1.0,3.0,1900-1959,22.0,2030.0,2227.0,12.0,2151,2239.0,48.0,48.0,1.0,3.0,2100-2159,0.0,NaN,0.0,97.0,91.0,57.0,1.0,350.0,2,0.0,0.0,0.0,0.0,48.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [7]:
print(df.shape)

(13453022, 110)


In [8]:
airport_coordinates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19148 entries, 0 to 19147
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   AIRPORT_ID                      19148 non-null  int64  
 1   AIRPORT                         19148 non-null  object 
 2   DISPLAY_AIRPORT_NAME            19148 non-null  object 
 3   DISPLAY_AIRPORT_CITY_NAME_FULL  19148 non-null  object 
 4   LAT_DEGREES                     19147 non-null  float64
 5   LAT_HEMISPHERE                  19147 non-null  object 
 6   LAT_MINUTES                     19147 non-null  float64
 7   LAT_SECONDS                     19147 non-null  float64
 8   LATITUDE                        19147 non-null  float64
 9   LON_DEGREES                     19147 non-null  float64
 10  LON_HEMISPHERE                  19147 non-null  object 
 11  LON_MINUTES                     19147 non-null  float64
 12  LON_SECONDS                     

In [9]:
airport_coordinates.head()

,AIRPORT_ID,AIRPORT,DISPLAY_AIRPORT_NAME,DISPLAY_AIRPORT_CITY_NAME_FULL,LAT_DEGREES,LAT_HEMISPHERE,LAT_MINUTES,LAT_SECONDS,LATITUDE,LON_DEGREES,LON_HEMISPHERE,LON_MINUTES,LON_SECONDS,LONGITUDE,AIRPORT_START_DATE,AIRPORT_THRU_DATE,AIRPORT_IS_CLOSED,AIRPORT_IS_LATEST
0,10001,01A,Afognak Lake Airport,"Afognak Lake, AK",58.0,N,6.0,34.0,58.109444,152.0,W,54.0,24.0,-152.906667,7/1/2007 12:00:00 AM,NaN,0,1
1,10003,03A,Bear Creek Mining Strip,"Granite Mountain, AK",65.0,N,32.0,53.0,65.548056,161.0,W,4.0,18.0,-161.071667,7/1/2007 12:00:00 AM,NaN,0,1
2,10004,04A,Lik Mining Camp,"Lik, AK",68.0,N,5.0,0.0,68.083333,163.0,W,10.0,0.0,-163.166667,7/1/2007 12:00:00 AM,NaN,0,1
3,10005,05A,Little Squaw Airport,"Little Squaw, AK",67.0,N,34.0,12.0,67.570000,148.0,W,11.0,2.0,-148.183889,8/1/2007 12:00:00 AM,NaN,0,1
4,10006,06A,Kizhuyak Bay,"Kizhuyak, AK",57.0,N,44.0,43.0,57.745278,152.0,W,52.0,58.0,-152.882778,10/1/2007 12:00:00 AM,NaN,0,1


In [10]:
print(airport_coordinates.shape)

(19148, 18)


In [11]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17516 entries, 0 to 17515
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   FlightHour                 17516 non-null  object 
 1   HourlyAvgTemperature       17516 non-null  float64
 2   HourlyAvgDewPoint          17516 non-null  float64
 3   HourlyAvgRelativeHumidity  17516 non-null  float64
 4   HourlyAvgWindSpeed         17508 non-null  float64
 5   HourlyAvgVisibility        17515 non-null  float64
dtypes: float64(5), object(1)
memory usage: 821.2+ KB


In [12]:
weather.head()

,FlightHour,HourlyAvgTemperature,HourlyAvgDewPoint,HourlyAvgRelativeHumidity,HourlyAvgWindSpeed,HourlyAvgVisibility
0,2021-08-01 00,83.0,74.0,74.33,8.0,10.0
1,2021-08-01 01,83.0,74.0,74.33,9.0,10.0
2,2021-08-01 02,81.0,75.0,82.03,8.0,10.0
3,2021-08-01 03,81.0,74.0,79.06,8.0,10.0
4,2021-08-01 04,79.0,72.0,79.17,4.0,10.0


In [13]:
print(weather.shape)

(17516, 6)


In [14]:
monthly_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 4 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   FlightMonth                 24 non-null     object 
 1   MonthlyAvgTemperature       24 non-null     float64
 2   MonthlyAvgDewPoint          24 non-null     float64
 3   MonthlyAvgRelativeHumidity  24 non-null     float64
dtypes: float64(3), object(1)
memory usage: 896.0+ bytes


In [15]:
monthly_weather.head()

,FlightMonth,MonthlyAvgTemperature,MonthlyAvgDewPoint,MonthlyAvgRelativeHumidity
0,2021-08,78.67,69.68,75.58
1,2021-09,73.56,63.83,73.74
2,2021-10,66.32,57.69,75.78
3,2021-11,52.52,37.33,59.47
4,2021-12,56.46,48.80,77.46


In [16]:
print(monthly_weather.shape)

(24, 4)


## Feature Selection and Merging

Remove non delay related features and datapoints

In [17]:
# Drop the last 46 columns
df = df.iloc[:, :-46]

# Create a boolean mask to filter rows where "cancelled" or "diverted" is equal to one
mask = (df['Cancelled'] != 1) & (df['Diverted'] != 1)

# Apply the mask to the DataFrame to filter the rows
df_filtered = df[mask]

# df_filtered now contains rows where "cancelled" and "diverted" are not equal to one

Only use flights from ATL

In [18]:
# Filter for flights with 10397 as the origin or destination id
atl_origin_id = 10397
df_filtered = df_filtered[(df_filtered['OriginAirportID'] == atl_origin_id) | (df_filtered['DestAirportID'] == atl_origin_id)]
df_filtered.reset_index(drop=True, inplace=True)

df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1276177 entries, 0 to 1276176
Data columns (total 64 columns):
 #   Column                           Non-Null Count    Dtype  
---  ------                           --------------    -----  
 0   Year                             1276177 non-null  int64  
 1   Quarter                          1276177 non-null  int64  
 2   Month                            1276177 non-null  int64  
 3   DayofMonth                       1276177 non-null  int64  
 4   DayOfWeek                        1276177 non-null  int64  
 5   FlightDate                       1276177 non-null  object 
 6   Reporting_Airline                1276177 non-null  object 
 7   DOT_ID_Reporting_Airline         1276177 non-null  int64  
 8   IATA_CODE_Reporting_Airline      1276177 non-null  object 
 9   Tail_Number                      1276177 non-null  object 
 10  Flight_Number_Reporting_Airline  1276177 non-null  int64  
 11  OriginAirportID                  1276177 non-null 

Remove unneccesary columns

In [19]:
columns_to_remove = [
    "Quarter", "DOT_ID_Reporting_Airline", "IATA_CODE_Reporting_Airline",
    "OriginAirportSeqID", "OriginCityMarketID", "OriginCityName", "OriginState",
    "OriginStateFips", "OriginStateName", "OriginWac", "DestAirportSeqID",
    "DestCityMarketID", "DestCityName", "DestState", "DestStateFips",
    "DestStateName", "DestWac", "DepDelayMinutes", "ArrDelayMinutes",
    "FirstDepTime", "TotalAddGTime", "LongestAddGTime", "Cancelled", "Diverted", "CancellationCode", "Flights"
]

# Remove the specified columns and reset the index
df_filtered = df_filtered.drop(columns=columns_to_remove)

In [20]:
df_filtered.head()

,Year,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,OriginAirportID,Origin,DestAirportID,Dest,CRSDepTime,DepTime,DepDelay,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,CRSElapsedTime,ActualElapsedTime,AirTime,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2021,8,21,6,2021-08-21,OH,N541EA,5622,10397,ATL,11278,DCA,1630,1628.0,-2.0,0.0,-1.0,1600-1659,25.0,1653.0,1814.0,3.0,1830,1817.0,-13.0,0.0,-1.0,1800-1859,120.0,109.0,81.0,547.0,3,NaN,NaN,NaN,NaN,NaN
1,2021,8,28,6,2021-08-28,OH,N562NN,5622,10397,ATL,11278,DCA,1630,1624.0,-6.0,0.0,-1.0,1600-1659,18.0,1642.0,1756.0,3.0,1830,1759.0,-31.0,0.0,-2.0,1800-1859,120.0,95.0,74.0,547.0,3,NaN,NaN,NaN,NaN,NaN
2,2021,8,21,6,2021-08-21,OH,N541EA,5622,11278,DCA,10397,ATL,1350,1348.0,-2.0,0.0,-1.0,1300-1359,30.0,1418.0,1535.0,7.0,1547,1542.0,-5.0,0.0,-1.0,1500-1559,117.0,114.0,77.0,547.0,3,NaN,NaN,NaN,NaN,NaN
3,2021,8,28,6,2021-08-28,OH,N562NN,5622,11278,DCA,10397,ATL,1350,1344.0,-6.0,0.0,-1.0,1300-1359,17.0,1401.0,1525.0,8.0,1547,1533.0,-14.0,0.0,-1.0,1500-1559,117.0,109.0,84.0,547.0,3,NaN,NaN,NaN,NaN,NaN
4,2021,8,21,6,2021-08-21,OO,N144SY,5274,12266,IAH,10397,ATL,950,947.0,-3.0,0.0,-1.0,0900-0959,17.0,1004.0,1244.0,10.0,1256,1254.0,-2.0,0.0,-1.0,1200-1259,126.0,127.0,100.0,689.0,3,NaN,NaN,NaN,NaN,NaN


Add Season

In [21]:
# Define a function to map month numbers to seasons
def map_to_season(month):
    if month in [12, 1, 2]:
        return 1 # Winter
    elif month in [3, 4, 5]:
        return 2 # Spring
    elif month in [6, 7, 8]:
        return 3 # Summer
    elif month in [9, 10, 11]:
        return 4 # Autumn
    else:
        return 0 # Unknown

# Create a new 'Season' column
df_filtered['Season'] = df_filtered['Month'].apply(map_to_season)

In [22]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1276177 entries, 0 to 1276176
Data columns (total 39 columns):
 #   Column                           Non-Null Count    Dtype  
---  ------                           --------------    -----  
 0   Year                             1276177 non-null  int64  
 1   Month                            1276177 non-null  int64  
 2   DayofMonth                       1276177 non-null  int64  
 3   DayOfWeek                        1276177 non-null  int64  
 4   FlightDate                       1276177 non-null  object 
 5   Reporting_Airline                1276177 non-null  object 
 6   Tail_Number                      1276177 non-null  object 
 7   Flight_Number_Reporting_Airline  1276177 non-null  int64  
 8   OriginAirportID                  1276177 non-null  int64  
 9   Origin                           1276177 non-null  object 
 10  DestAirportID                    1276177 non-null  int64  
 11  Dest                             1276177 non-null 

Add weather features for ATL

In [23]:
df_filtered['DepTime'] = df_filtered['DepTime'].astype(int)
df_filtered['DepTime'] = df_filtered['DepTime'].replace(2400, 0)

df_filtered = df_filtered.dropna(subset=['ArrTime'])
df_filtered['ArrTime'] = df_filtered['ArrTime'].astype(int)
df_filtered['ArrTime'] = df_filtered['ArrTime'].replace(2400, 0)

df_filtered['FlightDateTimeDep'] = df_filtered['FlightDate'] + ' ' + df_filtered['DepTime'].astype(str).str.zfill(4)
df_filtered['FlightDateTimeDep'] = pd.to_datetime(df_filtered['FlightDateTimeDep']).dt.floor('H').dt.strftime('%Y-%m-%d %H')

df_filtered['FlightDateTimeArr'] = df_filtered['FlightDate'] + ' ' + df_filtered['ArrTime'].astype(str).str.zfill(4)
df_filtered['FlightDateTimeArr'] = pd.to_datetime(df_filtered['FlightDateTimeArr']).dt.floor('H').dt.strftime('%Y-%m-%d %H')

# Merge on Departure Time
df_filtered_dep = pd.merge(df_filtered, weather, left_on='FlightDateTimeDep', right_on='FlightHour', how='inner')
df_filtered_dep.rename(columns={'FlightHour': 'FlightHourDep',
                                'HourlyAvgTemperature': 'HourlyAvgTemperatureDep',
                                'HourlyAvgDewPoint': 'HourlyAvgDewPointDep',
                                'HourlyAvgRelativeHumidity': 'HourlyAvgRelativeHumidityDep',
                                'HourlyAvgWindSpeed': 'HourlyAvgWindSpeedDep',
                                'HourlyAvgVisibility': 'HourlyAvgVisibilityDep'}, 
                       inplace=True)

# Merge on Arrival Time
df_filtered_arr = pd.merge(df_filtered, weather, left_on='FlightDateTimeArr', right_on='FlightHour', how='inner')
df_filtered_arr.rename(columns={'FlightHour': 'FlightHourArr',
                                'HourlyAvgTemperature': 'HourlyAvgTemperatureArr',
                                'HourlyAvgDewPoint': 'HourlyAvgDewPointArr',
                                'HourlyAvgRelativeHumidity': 'HourlyAvgRelativeHumidityArr',
                                'HourlyAvgWindSpeed': 'HourlyAvgWindSpeedArr',
                                'HourlyAvgVisibility': 'HourlyAvgVisibilityArr'},
                       inplace=True)

# Combine the results
df_filtered = pd.concat([df_filtered_dep, df_filtered_arr], axis=1)

# Remove duplicate columns (if any)
df_filtered = df_filtered.loc[:,~df_filtered.columns.duplicated()]

# Remove duplicate rows
df_filtered = df_filtered.drop_duplicates()

# Drop unnecessary columns
df_filtered = df_filtered.drop(['FlightDateTimeDep', 'FlightDateTimeArr'], axis=1)

In [24]:
df_filtered.info() # previous 1276025 rows , 56 columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1276025 entries, 0 to 1276024
Data columns (total 51 columns):
 #   Column                           Non-Null Count    Dtype  
---  ------                           --------------    -----  
 0   Year                             1276025 non-null  int64  
 1   Month                            1276025 non-null  int64  
 2   DayofMonth                       1276025 non-null  int64  
 3   DayOfWeek                        1276025 non-null  int64  
 4   FlightDate                       1276025 non-null  object 
 5   Reporting_Airline                1276025 non-null  object 
 6   Tail_Number                      1276025 non-null  object 
 7   Flight_Number_Reporting_Airline  1276025 non-null  int64  
 8   OriginAirportID                  1276025 non-null  int64  
 9   Origin                           1276025 non-null  object 
 10  DestAirportID                    1276025 non-null  int64  
 11  Dest                             1276025 non-null 

In [25]:
# Convert "FlightDate" to Date object
df_filtered['FlightDate'] = pd.to_datetime(df_filtered['FlightDate'])
monthly_weather['FlightMonth'] = pd.to_datetime(monthly_weather['FlightMonth'])

In [26]:
# combine date time to merge with monthly weather data
df_filtered['FlightMonth'] = df_filtered['FlightDate'].dt.to_period('M')
monthly_weather['FlightMonth'] = monthly_weather['FlightMonth'].dt.to_period('M')

# merge on FlightDateTime
df_filtered = pd.merge(df_filtered, monthly_weather[['FlightMonth', 'MonthlyAvgTemperature', 'MonthlyAvgDewPoint' , 'MonthlyAvgRelativeHumidity']], on='FlightMonth', how='left')

In [27]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1276025 entries, 0 to 1276024
Data columns (total 55 columns):
 #   Column                           Non-Null Count    Dtype         
---  ------                           --------------    -----         
 0   Year                             1276025 non-null  int64         
 1   Month                            1276025 non-null  int64         
 2   DayofMonth                       1276025 non-null  int64         
 3   DayOfWeek                        1276025 non-null  int64         
 4   FlightDate                       1276025 non-null  datetime64[ns]
 5   Reporting_Airline                1276025 non-null  object        
 6   Tail_Number                      1276025 non-null  object        
 7   Flight_Number_Reporting_Airline  1276025 non-null  int64         
 8   OriginAirportID                  1276025 non-null  int64         
 9   Origin                           1276025 non-null  object        
 10  DestAirportID                 

In [28]:
df_filtered.head()

,Year,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,OriginAirportID,Origin,DestAirportID,Dest,CRSDepTime,DepTime,DepDelay,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,CRSElapsedTime,ActualElapsedTime,AirTime,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,Season,FlightHourDep,HourlyAvgTemperatureDep,HourlyAvgDewPointDep,HourlyAvgRelativeHumidityDep,HourlyAvgWindSpeedDep,HourlyAvgVisibilityDep,FlightHourArr,HourlyAvgTemperatureArr,HourlyAvgDewPointArr,HourlyAvgRelativeHumidityArr,HourlyAvgWindSpeedArr,HourlyAvgVisibilityArr,FlightMonth,MonthlyAvgTemperature,MonthlyAvgDewPoint,MonthlyAvgRelativeHumidity
0,2021,8,21,6,2021-08-21,OH,N541EA,5622,10397,ATL,11278,DCA,1630,1628,-2.0,0.0,-1.0,1600-1659,25.0,1653.0,1814.0,3.0,1830,1817,-13.0,0.0,-1.0,1800-1859,120.0,109.0,81.0,547.0,3,NaN,NaN,NaN,NaN,NaN,3,2021-08-21 16,83.0,72.5,70.91,6.0,10.0,2021-08-21 18,84.0,72.0,67.28,7.0,10.0,2021-08,78.67,69.68,75.58
1,2021,8,21,6,2021-08-21,DL,N813NW,304,12173,HNL,10397,ATL,1630,1628,-2.0,0.0,-1.0,1600-1659,19.0,1647.0,704.0,7.0,714,711,-3.0,0.0,-1.0,0700-0759,524.0,523.0,497.0,4502.0,11,NaN,NaN,NaN,NaN,NaN,3,2021-08-21 16,83.0,72.5,70.91,6.0,10.0,2021-08-21 18,84.0,72.0,67.28,7.0,10.0,2021-08,78.67,69.68,75.58
2,2021,8,21,6,2021-08-21,DL,N370DN,333,10397,ATL,10721,BOS,1555,1608,13.0,0.0,0.0,1500-1559,10.0,1618.0,1820.0,8.0,1835,1828,-7.0,0.0,-1.0,1800-1859,160.0,140.0,122.0,946.0,4,NaN,NaN,NaN,NaN,NaN,3,2021-08-21 16,83.0,72.5,70.91,6.0,10.0,2021-08-21 18,84.0,72.0,67.28,7.0,10.0,2021-08,78.67,69.68,75.58
3,2021,8,21,6,2021-08-21,DL,N694DL,349,12441,JAC,10397,ATL,1407,1650,163.0,1.0,10.0,1400-1459,12.0,1702.0,2205.0,6.0,1940,2211,151.0,1.0,10.0,1900-1959,213.0,201.0,183.0,1572.0,7,151.0,0.0,0.0,0.0,0.0,3,2021-08-21 16,83.0,72.5,70.91,6.0,10.0,2021-08-21 18,84.0,72.0,67.28,7.0,10.0,2021-08,78.67,69.68,75.58
4,2021,8,21,6,2021-08-21,DL,N582NW,465,13487,MSP,10397,ATL,1515,1602,47.0,1.0,3.0,1500-1559,10.0,1612.0,1915.0,6.0,1845,1921,36.0,1.0,2.0,1800-1859,150.0,139.0,123.0,907.0,4,11.0,0.0,0.0,0.0,25.0,3,2021-08-21 16,83.0,72.5,70.91,6.0,10.0,2021-08-21 18,84.0,72.0,67.28,7.0,10.0,2021-08,78.67,69.68,75.58


Add Longitude and Latitude

In [29]:
print(airport_coordinates['AIRPORT_ID'].nunique())

6659


In [30]:
airport_coordinates = airport_coordinates[['AIRPORT_ID', 'LATITUDE', 'LONGITUDE']]

# Step 1: Group by Airport ID and aggregate coordinates
grouped_coordinates = airport_coordinates.groupby('AIRPORT_ID').agg({'LONGITUDE': 'mean', 'LATITUDE': 'mean'}).reset_index()

# Merge for Origin Airport
df_filtered = pd.merge(df_filtered, grouped_coordinates, left_on='OriginAirportID', right_on='AIRPORT_ID', how='left')
df_filtered.rename(columns={'LONGITUDE': 'OriginLongitude', 'LATITUDE': 'OriginLatitude'}, inplace=True)
df_filtered.drop(columns='AIRPORT_ID', inplace=True)

# Merge for Destination Airport
df_filtered = pd.merge(df_filtered, grouped_coordinates, left_on='DestAirportID', right_on='AIRPORT_ID', how='left')
df_filtered.rename(columns={'LONGITUDE': 'DestLongitude', 'LATITUDE': 'DestLatitude'}, inplace=True)
df_filtered.drop(columns='AIRPORT_ID', inplace=True)

# If you have duplicates in OriginAirportID and DestAirportID, the above merges might create duplicate rows.
# To keep all rows, you can drop duplicates based on the entire row, keeping the first occurrence.
df_filtered.drop_duplicates(inplace=True)

In [31]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1276025 entries, 0 to 1276024
Data columns (total 59 columns):
 #   Column                           Non-Null Count    Dtype         
---  ------                           --------------    -----         
 0   Year                             1276025 non-null  int64         
 1   Month                            1276025 non-null  int64         
 2   DayofMonth                       1276025 non-null  int64         
 3   DayOfWeek                        1276025 non-null  int64         
 4   FlightDate                       1276025 non-null  datetime64[ns]
 5   Reporting_Airline                1276025 non-null  object        
 6   Tail_Number                      1276025 non-null  object        
 7   Flight_Number_Reporting_Airline  1276025 non-null  int64         
 8   OriginAirportID                  1276025 non-null  int64         
 9   Origin                           1276025 non-null  object        
 10  DestAirportID                 

In [32]:
df_filtered.reset_index(drop=True, inplace=True)

## Confimation and Save Dataset

In [33]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1276025 entries, 0 to 1276024
Data columns (total 59 columns):
 #   Column                           Non-Null Count    Dtype         
---  ------                           --------------    -----         
 0   Year                             1276025 non-null  int64         
 1   Month                            1276025 non-null  int64         
 2   DayofMonth                       1276025 non-null  int64         
 3   DayOfWeek                        1276025 non-null  int64         
 4   FlightDate                       1276025 non-null  datetime64[ns]
 5   Reporting_Airline                1276025 non-null  object        
 6   Tail_Number                      1276025 non-null  object        
 7   Flight_Number_Reporting_Airline  1276025 non-null  int64         
 8   OriginAirportID                  1276025 non-null  int64         
 9   Origin                           1276025 non-null  object        
 10  DestAirportID                 

In [34]:
df_filtered.head()

,Year,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,OriginAirportID,Origin,DestAirportID,Dest,CRSDepTime,DepTime,DepDelay,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,CRSElapsedTime,ActualElapsedTime,AirTime,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,Season,FlightHourDep,HourlyAvgTemperatureDep,HourlyAvgDewPointDep,HourlyAvgRelativeHumidityDep,HourlyAvgWindSpeedDep,HourlyAvgVisibilityDep,FlightHourArr,HourlyAvgTemperatureArr,HourlyAvgDewPointArr,HourlyAvgRelativeHumidityArr,HourlyAvgWindSpeedArr,HourlyAvgVisibilityArr,FlightMonth,MonthlyAvgTemperature,MonthlyAvgDewPoint,MonthlyAvgRelativeHumidity,OriginLongitude,OriginLatitude,DestLongitude,DestLatitude
0,2021,8,21,6,2021-08-21,OH,N541EA,5622,10397,ATL,11278,DCA,1630,1628,-2.0,0.0,-1.0,1600-1659,25.0,1653.0,1814.0,3.0,1830,1817,-13.0,0.0,-1.0,1800-1859,120.0,109.0,81.0,547.0,3,NaN,NaN,NaN,NaN,NaN,3,2021-08-21 16,83.0,72.5,70.91,6.0,10.0,2021-08-21 18,84.0,72.0,67.28,7.0,10.0,2021-08,78.67,69.68,75.58,-84.427619,33.638452,-77.037611,38.851667
1,2021,8,21,6,2021-08-21,DL,N813NW,304,12173,HNL,10397,ATL,1630,1628,-2.0,0.0,-1.0,1600-1659,19.0,1647.0,704.0,7.0,714,711,-3.0,0.0,-1.0,0700-0759,524.0,523.0,497.0,4502.0,11,NaN,NaN,NaN,NaN,NaN,3,2021-08-21 16,83.0,72.5,70.91,6.0,10.0,2021-08-21 18,84.0,72.0,67.28,7.0,10.0,2021-08,78.67,69.68,75.58,-157.921167,21.318111,-84.427619,33.638452
2,2021,8,21,6,2021-08-21,DL,N370DN,333,10397,ATL,10721,BOS,1555,1608,13.0,0.0,0.0,1500-1559,10.0,1618.0,1820.0,8.0,1835,1828,-7.0,0.0,-1.0,1800-1859,160.0,140.0,122.0,946.0,4,NaN,NaN,NaN,NaN,NaN,3,2021-08-21 16,83.0,72.5,70.91,6.0,10.0,2021-08-21 18,84.0,72.0,67.28,7.0,10.0,2021-08,78.67,69.68,75.58,-84.427619,33.638452,-71.006250,42.363333
3,2021,8,21,6,2021-08-21,DL,N694DL,349,12441,JAC,10397,ATL,1407,1650,163.0,1.0,10.0,1400-1459,12.0,1702.0,2205.0,6.0,1940,2211,151.0,1.0,10.0,1900-1959,213.0,201.0,183.0,1572.0,7,151.0,0.0,0.0,0.0,0.0,3,2021-08-21 16,83.0,72.5,70.91,6.0,10.0,2021-08-21 18,84.0,72.0,67.28,7.0,10.0,2021-08,78.67,69.68,75.58,-110.737361,43.606944,-84.427619,33.638452
4,2021,8,21,6,2021-08-21,DL,N582NW,465,13487,MSP,10397,ATL,1515,1602,47.0,1.0,3.0,1500-1559,10.0,1612.0,1915.0,6.0,1845,1921,36.0,1.0,2.0,1800-1859,150.0,139.0,123.0,907.0,4,11.0,0.0,0.0,0.0,25.0,3,2021-08-21 16,83.0,72.5,70.91,6.0,10.0,2021-08-21 18,84.0,72.0,67.28,7.0,10.0,2021-08,78.67,69.68,75.58,-93.219722,44.884028,-84.427619,33.638452


In [35]:
# Check if there are any duplicate rows in the entire DataFrame
has_duplicates = df_filtered.duplicated().any()

# Display the result
print("Does the DataFrame have any duplicate rows?", has_duplicates)

Does the DataFrame have any duplicate rows? False


In [36]:
df_filtered.to_csv('../data/cleaned/atl_cleaned.csv', index=False)  # Set index to False to avoid saving the index as a column

## Atlanta Airport

In [37]:
# # Filter for flights with 10397 as the origin or destination id
# atl_origin_id = 10397
# df_atl = df_filtered[(df_filtered['OriginAirportID'] == atl_origin_id) | (df_filtered['DestAirportID'] == atl_origin_id)]
# df_atl.reset_index(drop=True, inplace=True)

# df_atl.info()

In [38]:
# df_atl.head(10)

In [39]:
# df_atl.to_csv('../data/cleaned/atl_cleaned.csv', index=False)

## Top 5 Airports

In [40]:
# # List of top ten airport codes to filter for
# airport_codes = ['ATL', 'DFW', 'DEN', 'ORD', 'LAX']

# # Create a DataFrame containing 'Origin' values
# origin_df = pd.DataFrame({'Origin': airport_codes})

# # Merge the 'Origin' DataFrame with the original DataFrame to get 'OriginAirportID'
# result_df = pd.merge(origin_df, df_filtered[['Origin', 'OriginAirportID']], on='Origin', how='left')

# # Extract only the unique values of the 'OriginAirportID' column
# airport_ids = result_df['OriginAirportID'].unique()

# result_df.info()
# print(airport_ids)

In [41]:
# # Use the isin() method to filter the DataFrame based on the list of airport ids
# df_top_ten = df_filtered[(df_filtered['OriginAirportID'].isin(airport_ids)) | (df_filtered['DestAirportID'].isin(airport_ids))]

# df_top_ten.info()

## Save Cleaned Data

In [42]:
# # Save data to .csv files
# df_filtered.to_csv('../data/cleaned/flight_data_cleaned.csv', index=False)  # Set index to False to avoid saving the index as a column
# df_atl.to_csv('../data/cleaned/atl_cleaned.csv', index=False)
# df_top_ten.to_csv('../data/cleaned/top_five_cleaned.csv', index=False)